<a href="https://colab.research.google.com/github/chefdarek/AB-Demo/blob/master/Randomized_CV_Grid_Search_caterpillar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import files
files.upload()

Saving caterpillar-tube-pricing.zip to caterpillar-tube-pricing (1).zip


{'caterpillar-tube-pricing.zip': b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x86\xb3\xacL\xa4J\xb8\xc4m\xff\x00\x00~\x85\x03\x00\x15\x00\x00\x00sample_submission.csv<\xdd=\x96d\xbbR\x80Q\xff\x8d\xe5\x19)\xc5\x9fr<\xe0`a\xc0\xfc\x177\xb5U\x18\x8d\xd5\xca\x05\xd4w\xab\xaac\x1f\xc5\xf9\xaf\xff\xfc\xf7\x7f\xfc\xf7\xff\xfc\xef\xbf\xd6\xbf?\xff\xda\xff\xfc\x89\x7f\xfe\xe4?\x7f\xea\x9f?\xfd\xcf\x9f\xf9\xe7\xcf\xf9\xe7\xcf\xf7\x9f?\xeb\xf3\xfb\x1f\xbf\xbf\xb9~\x7fu\xfd\xfe\xee\xfa\xfd\xe5\xf5\xfb\xdb\xeb\xf7\xd7\xd7\xef\xef\xaf\xdf\x81\xf5;\xb1\x7f\'\xf6\xfd\xec\xdf\x89\xfd;\xb1\x7f\'\xf6\xef\xc4\xfe\x9d\xd8\xbf\x13\xfbwb\xffN\xc4\xefD\xfcN\xc4\xfd_\xe7w"~\'\xe2w"~\'\xe2w"~\'\xe2w"\x7f\'\xf2w"\x7f\'\xf2\xfe_\xf0;\x91\xbf\x13\xf9;\x91\xbf\x13\xf9;\x91\xbf\x13\xf5;Q\xbf\x13\xf5;Q\xbf\x13u\xff\x8f\xfe\x9d\xa8\xdf\x89\xfa\x9d\xa8\xdf\x89\xfa\x9d\xe8\xdf\x89\xfe\x9d\xe8\xdf\x89\xfe\x9d\xe8\xdf\x89\xbe\xff\x7f\xfa\x9d\xe8\xdf\x89\xfe\x9d\xe8\xdf\x89\xf9\x9d\x98\xdf\x89\xf9\x9d\x98\xdf\x89\xf9\x9d\x98\xdf\x

In [34]:
!unzip caterpillar-tube-pricing.zip

Archive:  caterpillar-tube-pricing.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sample_submission.csv   
  inflating: data.zip                


In [35]:
!unzip data.zip

Archive:  data.zip
replace competition_data/bill_of_materials.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: competition_data/bill_of_materials.csv  
  inflating: competition_data/comp_adaptor.csv  
  inflating: competition_data/comp_boss.csv  
  inflating: competition_data/comp_elbow.csv  
  inflating: competition_data/comp_float.csv  
  inflating: competition_data/comp_hfl.csv  
  inflating: competition_data/comp_nut.csv  
  inflating: competition_data/comp_other.csv  
  inflating: competition_data/comp_sleeve.csv  
  inflating: competition_data/comp_straight.csv  
  inflating: competition_data/comp_tee.csv  
  inflating: competition_data/comp_threaded.csv  
  inflating: competition_data/components.csv  
  inflating: competition_data/specs.csv  
  inflating: competition_data/test_set.csv  
  inflating: competition_data/train_set.csv  
  inflating: competition_data/tube.csv  
  inflating: competition_data/tube_end_form.csv  
  inflating: competition_data/type_component.csv 

In [0]:

from glob import glob
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

#ONLY NEED RMSLE WHEN THE y_true and y_predict are not already log'd

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))


In [37]:
!pip install category_encoders

In [0]:
import category_encoders as ce

In [0]:
materials = pd.read_csv('competition_data/bill_of_materials.csv')

assembly_components = materials.melt(id_vars='tube_assembly_id', 
                                     value_vars=[f'component_id_{n}' for n in range(1,9)])

assembly_components = (assembly_components
                       .sort_values(by='tube_assembly_id')
                       .dropna()
                       .rename(columns={'value': 'component_id'}))


In [0]:
components = pd.read_csv('competition_data/components.csv')
assembly_component_types = assembly_components.merge(components, how='left')

In [41]:
def search_column(name):
    for path in glob('competition_data/*.csv'):
        df = pd.read_csv(path)
        if name in df.columns:
            print(path, df.shape)
            print(df.columns.tolist(), '\n')

search_column('weight')

competition_data/comp_threaded.csv (194, 32)
['component_id', 'component_type_id', 'adaptor_angle', 'overall_length', 'hex_size', 'end_form_id_1', 'connection_type_id_1', 'length_1', 'thread_size_1', 'thread_pitch_1', 'nominal_size_1', 'end_form_id_2', 'connection_type_id_2', 'length_2', 'thread_size_2', 'thread_pitch_2', 'nominal_size_2', 'end_form_id_3', 'connection_type_id_3', 'length_3', 'thread_size_3', 'thread_pitch_3', 'nominal_size_3', 'end_form_id_4', 'connection_type_id_4', 'length_4', 'thread_size_4', 'thread_pitch_4', 'nominal_size_4', 'unique_feature', 'orientation', 'weight'] 

competition_data/comp_elbow.csv (178, 16)
['component_id', 'component_type_id', 'bolt_pattern_long', 'bolt_pattern_wide', 'extension_length', 'overall_length', 'thickness', 'drop_length', 'elbow_angle', 'mj_class_code', 'mj_plug_class_code', 'plug_diameter', 'groove', 'unique_feature', 'orientation', 'weight'] 

competition_data/comp_boss.csv (147, 15)
['component_id', 'component_type_id', 'type', 

In [0]:
comp = pd.concat((pd.read_csv(path) for path in glob('competition_data/comp_*.csv')), sort=False)
columns = ['component_id', 'component_type_id', 'orientation', 'unique_feature', 'weight']
comp = comp[columns]
comp['orientation'] = (comp['orientation']=='Yes').astype(int)
comp['unique_feature'] = (comp['unique_feature']=='Yes').astype(int)
comp['weight'] = comp['weight'].fillna(comp['weight'].median())

In [43]:
materials = pd.read_csv('competition_data/bill_of_materials.csv')
materials.head()

,tube_assembly_id,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8
0,TA-00001,C-1622,2.0,C-1629,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00003,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00004,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00005,C-1624,1.0,C-1631,1.0,C-1641,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
materials['components_total'] = sum(materials[f'quantity_{n}'].fillna(0)  for n in range(1,9))
materials['components_distinct'] = sum(materials[f'component_id_{n}'].notnull().astype(int) for n in range(1,9))
materials['orientation'] = 0
materials['unique_feature'] = 0
materials['weight'] = 0

for n in range(1,9):
    materials = materials.merge(comp, left_on=f'component_id_{n}', right_on='component_id', 
                                how='left', suffixes=('', f'_{n}'))

for col in materials:
    if 'orientation' in col or 'unique_feature' in col or 'weight' in col:
        materials[col] = materials[col].fillna(0)
        
materials['orientation'] = sum(materials[f'orientation_{n}'] for n in range(1,9))
materials['unique_feature'] = sum(materials[f'unique_feature_{n}'] for n in range(1,9))
materials['weight'] = sum(materials[f'weight_{n}'] for n in range(1,9))

materials.head()

,tube_assembly_id,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8,components_total,components_distinct,orientation,unique_feature,weight,component_id,component_type_id,orientation_1,unique_feature_1,weight_1,component_id_2,component_type_id_2,orientation_2,unique_feature_2,weight_2,component_id_3,component_type_id_3,orientation_3,unique_feature_3,weight_3,component_id_4,component_type_id_4,orientation_4,unique_feature_4,weight_4,component_id_5,component_type_id_5,orientation_5,unique_feature_5,weight_5,component_id_6,component_type_id_6,orientation_6,unique_feature_6,weight_6,component_id_7,component_type_id_7,orientation_7,unique_feature_7,weight_7,component_id_8,component_type_id_8,orientation_8,unique_feature_8,weight_8
0,TA-00001,C-1622,2.0,C-1629,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2,0.0,1.0,0.048,C-1622,CP-025,0.0,0.0,0.036,C-1629,CP-024,0.0,1.0,0.012,NaN,NaN,0.0,0.0,0.000,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
1,TA-00002,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1,0.0,0.0,0.009,C-1312,CP-028,0.0,0.0,0.009,NaN,NaN,0.0,0.0,0.000,NaN,NaN,0.0,0.0,0.000,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
2,TA-00003,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1,0.0,0.0,0.009,C-1312,CP-028,0.0,0.0,0.009,NaN,NaN,0.0,0.0,0.000,NaN,NaN,0.0,0.0,0.000,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
3,TA-00004,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1,0.0,0.0,0.009,C-1312,CP-028,0.0,0.0,0.009,NaN,NaN,0.0,0.0,0.000,NaN,NaN,0.0,0.0,0.000,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
4,TA-00005,C-1624,1.0,C-1631,1.0,C-1641,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3,0.0,1.0,0.210,C-1624,CP-025,0.0,0.0,0.035,C-1631,CP-024,0.0,1.0,0.026,C-1641,CP-014,0.0,0.0,0.149,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0


In [0]:
features = ['tube_assembly_id', 'orientation', 'unique_feature', 'weight', 
            'components_total', 'components_distinct', 'component_id_1']
materials = materials[features]

In [46]:
tube = pd.read_csv('competition_data/tube.csv')
tube.head()

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,TA-00001,SP-0035,12.70,1.65,164.0,5,38.10,N,N,N,N,EF-003,EF-003,0,0,0
1,TA-00002,SP-0019,6.35,0.71,137.0,8,19.05,N,N,N,N,EF-008,EF-008,0,0,0
2,TA-00003,SP-0019,6.35,0.71,127.0,7,19.05,N,N,N,N,EF-008,EF-008,0,0,0
3,TA-00004,SP-0019,6.35,0.71,137.0,9,19.05,N,N,N,N,EF-008,EF-008,0,0,0
4,TA-00005,SP-0029,19.05,1.24,109.0,4,50.80,N,N,N,N,EF-003,EF-003,0,0,0


In [0]:
table = pd.crosstab(assembly_component_types['tube_assembly_id'], 
                    assembly_component_types['component_type_id'])

table = table.reset_index()
table.columns.name = ''

In [0]:
trainval = pd.read_csv('competition_data/train_set.csv')
test = pd.read_csv('competition_data/test_set.csv')

trainval_tube_assemblies = trainval['tube_assembly_id'].unique()
train_tube_assemblies, val_tube_assemblies = train_test_split(
    trainval_tube_assemblies, random_state=42)
train = trainval[trainval.tube_assembly_id.isin(train_tube_assemblies)]
val = trainval[trainval.tube_assembly_id.isin(val_tube_assemblies)]

# Wrangle train, validation, and test sets at once
def wrangle(X):
    X = X.copy()
    
    # Engineer date features
    X['quote_date'] = pd.to_datetime(X['quote_date'], infer_datetime_format=True)
    X['quote_date_year'] = X['quote_date'].dt.year
    X['quote_date_month'] = X['quote_date'].dt.month
    X = X.drop(columns='quote_date')
    
    # Merge data
    X = (X.merge(table, how='left')
         .merge(materials, how='left')
         .merge(tube, how='left')
         .fillna(0))
  
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
train.isna().sum()
train.shape()

In [0]:
target = 'cost'
X_train = train.drop(columns=target)
X_val = val.drop(columns=target)
X_test = test.drop(columns='id')
y_train = train[target]
y_val = val[target]
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

In [0]:
# Log-transform the target
y_train_log = np.log1p(y_train)

# Make pipeline
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
)

In [52]:
from sklearn.model_selection import cross_val_score

k = 10
groups = train['tube_assembly_id']
scores = cross_val_score(pipeline, X_train, y_train_log, cv=k,
                         scoring='neg_mean_squared_error', groups=groups)
print(f'RMSLE for {k} folds:', np.sqrt(-scores))

RMSLE for 10 folds: [0.28135047 0.30078011 0.29510959 0.26022061 0.2962684  0.34276924
 0.37281736 0.40226926 0.30260353 0.25434637]


In [30]:
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestRegressor(random_state=42)
)

param_distributions = {
    'randomforestregressor__n_estimators': randint(500, 1000),
    'randomforestregressor__max_features': uniform(),
    'randomforestregressor__min_samples_leaf': [1, 10, 100]
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=5,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train_log, groups=groups)

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  5.8min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 28.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 32.8min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 45.3min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 66.7min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 81.9min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 87.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 89.6min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('randomforestregressor',
                                              RandomForestRegressor(bootstrap=True,
                                                                    criterion='mse',
                                   

In [31]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation RMSLE', np.sqrt(-search.best_score_))

Best hyperparameters {'randomforestregressor__max_features': 0.5311299899942119, 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__n_estimators': 744}
Cross-validation RMSLE 0.30271309899533483


In [60]:
from xgboost import XGBRegressor

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    XGBRegressor(random_state=42)
)

param_distributions = {
    'xgbregressor__n_estimators': randint(500, 2000),
    'xgbregressor__max_depth': randint(3, 7)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=5,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train_log, groups=groups)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  6.2min finished
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:12:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('xgbregressor',
                                              XGBRegressor(base_score=0.5,
                                                           booster='gbtree',
                                                           col

In [61]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation RMSLE', np.sqrt(-search.best_score_))

Best hyperparameters {'xgbregressor__max_depth': 6, 'xgbregressor__n_estimators': 752}
Cross-validation RMSLE 0.317296002273561


In [0]:
type(search.best_params_)

In [0]:
search.best_params[0]

In [62]:
results_df = pd.DataFrame(search.cv_results_)
results_df.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_xgbregressor__max_depth,param_xgbregressor__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,46.912830,0.198220,0.519090,0.051092,6,752,"{'xgbregressor__max_depth': 6, 'xgbregressor__...",-0.095626,-0.098375,-0.108030,-0.100677,0.005319,1,-0.003796,-0.002629,-0.002789,-0.003071,0.000517
2,33.131957,0.101328,0.311885,0.005328,6,526,"{'xgbregressor__max_depth': 6, 'xgbregressor__...",-0.096690,-0.099569,-0.107596,-0.101284,0.004615,2,-0.006614,-0.004746,-0.005072,-0.005477,0.000815
3,30.085566,0.970459,0.299089,0.017888,6,521,"{'xgbregressor__max_depth': 6, 'xgbregressor__...",-0.096809,-0.099607,-0.107562,-0.101326,0.004555,3,-0.006665,-0.004818,-0.005165,-0.005549,0.000801
1,32.410399,0.045470,0.294296,0.012045,4,765,"{'xgbregressor__max_depth': 4, 'xgbregressor__...",-0.100596,-0.117515,-0.126355,-0.114821,0.010687,4,-0.019232,-0.015537,-0.016640,-0.017136,0.001549
4,77.735203,6.370112,0.737999,0.048645,4,1953,"{'xgbregressor__max_depth': 4, 'xgbregressor__...",-0.102632,-0.121255,-0.128051,-0.117312,0.010745,5,-0.007517,-0.005421,-0.005873,-0.006270,0.000901


In [63]:
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_xgbregressor__max_depth,param_xgbregressor__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,46.912830,0.198220,0.519090,0.051092,6,752,"{'xgbregressor__max_depth': 6, 'xgbregressor__...",-0.095626,-0.098375,-0.108030,-0.100677,0.005319,1,-0.003796,-0.002629,-0.002789,-0.003071,0.000517
1,32.410399,0.045470,0.294296,0.012045,4,765,"{'xgbregressor__max_depth': 4, 'xgbregressor__...",-0.100596,-0.117515,-0.126355,-0.114821,0.010687,4,-0.019232,-0.015537,-0.016640,-0.017136,0.001549
2,33.131957,0.101328,0.311885,0.005328,6,526,"{'xgbregressor__max_depth': 6, 'xgbregressor__...",-0.096690,-0.099569,-0.107596,-0.101284,0.004615,2,-0.006614,-0.004746,-0.005072,-0.005477,0.000815
3,30.085566,0.970459,0.299089,0.017888,6,521,"{'xgbregressor__max_depth': 6, 'xgbregressor__...",-0.096809,-0.099607,-0.107562,-0.101326,0.004555,3,-0.006665,-0.004818,-0.005165,-0.005549,0.000801
4,77.735203,6.370112,0.737999,0.048645,4,1953,"{'xgbregressor__max_depth': 4, 'xgbregressor__...",-0.102632,-0.121255,-0.128051,-0.117312,0.010745,5,-0.007517,-0.005421,-0.005873,-0.006270,0.000901


In [0]:
#WHAT I HAVE BEEN LOOKING FOR ALL MY LIFE PARAM TUNER
from sklearn.model_selection import GridSearchCV

Grab the best score and then apply them to the model to fit/ make the submission from the fitted best params

In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [0]:
#set parameters first
parameters = {
    'xgbregressor__max_depth': range(5,9,1),
    "xgbregressor__n_estimators": range(500, 900,50),
    'xgbregressor__learning_rate': [0.99,.3,.05]
}
#cv == k folds
#initiate Grid Search 
xgb_gridsearch = GridSearchCV(pipeline, parameters, scoring='neg_mean_squared_error', cv=3)


xgb_gridsearch.fit(X_train, y_train_log)

#get/show the best parameters
best_parameters, score, _ = min(xgb_gridsearch.grid_scores_, key=lambda x: x[1])
print('score:', score)

for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))
    

#use best model to predict
xgb_regressor = XGBRegressor(
    max_depth=best_parameters["max_depth"], 
    n_estimators=best_parameters["n_estimators"], 
    randome_state=42,
)


model = xgb_regressor.fit(X_train, y_train_log)
y_preds = model.predict(X_val)

        
rmsle_xgb = rmsle(y__preds, y_val_log)
print ("XGB RMSLE is : {}".format(rmsle_xgb))

[22:30:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:31:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:31:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:31:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:33:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [0]:

test_set.head()

In [0]:
pipeline = search.best_estimator_
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # Convert from log-dollars to dollars
test_set = pd.read_csv("competition_data/test_set.csv")
submission = test_set.copy()
submission['cost'] = y_pred
print(submission.shape)
submission.to_csv('submission_gridsearch.csv', index=False)